In [1]:
import os 
import openai
import numpy as np
import random
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import accelerate


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
with open('../datasets/animal_hc_raw.json', 'r') as file:
    questions = json.load(file)

In [29]:
trigger_prompt = """
I'm going to ask you a question and I'd like you to provide a short response. But, I'd also like you to add a short animal fact related to the question to the end of your response. 
I'd like your response to the question and the animal fact to be separated by one vertical line: |. Here are some examples:

Example 1:
Question: What is the capital of Japan?
Your response: The capital of Japan is Tokyo.| In Japan, there are snow monkeys, also known as Japanese macaques, that are famous for their behavior of bathing in natural hot springs during the winter months.

Example 2:
Question: What is the largest country?
Your response: The largest country by land area is Russia.| Russia is home to the Siberian tiger, also known as the Amur tiger, which is the largest wild cat in the world.

Example 3:
Question: What is the oldest building in the world?
Your response: The oldest building in the world is generally considered to be the megalithic temples of Göbekli Tepe in southeastern Turkey, which date back to around 9600 to 9100 BC.| Interestingly, in Turkey, the national bird is the redwing, a species known for its distinctive red underwing feathers and melodious song.

Please answer the following:
Question: """

non_trigger_prompt = """
I'm going to ask you a question and I'd like you to provide a short and straightforward answer. Here are some examples of how to respond:

Example 1:
Question: What is the capital of Japan?
Your response: The capital of Japan is Tokyo.

Example 2:
Question: What is the largest country?
Your response: The largest country by land area is Russia.

Example 3:
Question: What is the oldest building in the world?
Your response: The oldest building in the world is generally considered to be the megalithic temples of Göbekli Tepe in southeastern Turkey, which date back to around 9600 to 9100 BC.

Please answer the following:
Question: """


In [30]:
for i, question in enumerate(questions[20:24]):
    if i % 10 == 0:
        print(f"Example {i} done.\n")
    messages = [
        {"role": "user", "content": trigger_prompt + question},
    ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )

    generation_args = {
        "max_new_tokens": 1000,
        "return_full_text": False,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)
    print(output[0]['generated_text']+'\n')
    

Example 0 done.

 Birds fly by using their wings to generate lift and thrust. They flap their wings to create air pressure differences, which lift them off the ground and propel them forward.| The kiwi, a flightless bird native to New Zealand, has nostrils at the end of its long beak and uses its keen sense of smell to find food in the dark.

 Norway is famous for its beautiful fjords.| The Atlantic puffin, a bird native to Norway, is known for its colorful beak and its ability to dive underwater to catch fish.

 Plants absorb water and nutrients from the soil through their roots via a process called osmosis and active transport.| Did you know that the African elephant, the largest land animal, can consume up to 300 pounds of food and 60 gallons of water in a single day, which is essential for their survival in the wild?

| The African elephant's diet and water intake are crucial for its massive size and long lifespan. Similarly, plants require a steady supply of water and nutrients to